<a href="https://colab.research.google.com/github/paramrajyadav/Chat_with_PDF_FAISS/blob/main/Chat_with_PDF_FAISS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [34]:

!pip install langchain openai tiktoken rapidocr-onnxruntime
!pip install faiss-cpu
!pip install langchain-community
!pip install PyPDF2
!pip install unstructured[pdf]
!pip install langchain
!pip install unstructured
!pip install openai
!pip install Cython
!pip install tiktoken

!pip install --upgrade langchain-astradb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.4/124.4 kB 5.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.9/44.9 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.9/18.9 MB 58.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 kB 8.2 MB/s eta 0:00:00
  Created wheel for bson: filename=bson-0.5.10-py3-none-any.whl size=11976 sha256=3157b4868582f53385cd264fb060bb0947c04917e688f216d67622f84348802c
  Stored in directory: /root/.cache/pip/wheels/36/49/3b/8b33954dfae7a176009c4d721a45af56c8a9c1cdc3ee947945
Successfully built bson


In [35]:
import requests
from langchain.document_loaders import TextLoader
from langchain.vectorstores import FAISS

import os
from getpass import getpass

#     load_dataset,
# )from datasets import (

from langchain_community.document_loaders import PyPDFLoader
from langchain_core.documents import Document
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough

from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.document_loaders import UnstructuredPDFLoader
from langchain.indexes import VectorstoreIndexCreator

In [36]:

from google.colab import userdata
OPENAI_API_KEY=userdata.get('OPENAI_API_KEY')

In [37]:

import os
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

In [10]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [46]:
pdf_folder_path="/content/drive/MyDrive/Dataset/text"

In [47]:
os.listdir(pdf_folder_path)

['state_of_the_union.txt']

In [59]:
pdf_file_path=os.path.join(pdf_folder_path,"state_of_the_union.txt")

In [60]:

loader1=TextLoader(pdf_file_path)

In [61]:
document=loader1.load()

In [63]:
print(document[0].page_content)

 Madam Speaker, Madam Vice President, our First Lady and Second Gentleman. Members of Congress and the Cabinet. Justices of the Supreme Court. My fellow Americans.  

Last year COVID-19 kept us apart. This year we are finally together again. 

Tonight, we meet as Democrats Republicans and Independents. But most importantly as Americans. 

With a duty to one another to the American people to the Constitution. 

And with an unwavering resolve that freedom will always triumph over tyranny. 

Six days ago, Russia’s Vladimir Putin sought to shake the foundations of the free world thinking he could make it bend to his menacing ways. But he badly miscalculated. 

He thought he could roll into Ukraine and the world would roll over. Instead he met a wall of strength he never imagined. 

He met the Ukrainian people. 

From President Zelenskyy to every Ukrainian, their fearlessness, their courage, their determination, inspires the world. 

Groups of citizens blocking tanks with their bodies. Ever

In [64]:

from langchain.text_splitter import RecursiveCharacterTextSplitter

In [65]:
text_splitter=RecursiveCharacterTextSplitter(chunk_size=500,chunk_overlap=50)

In [66]:
text_chunks=text_splitter.split_documents(document)

In [67]:
text_chunks

[Document(page_content='Madam Speaker, Madam Vice President, our First Lady and Second Gentleman. Members of Congress and the Cabinet. Justices of the Supreme Court. My fellow Americans.  \n\nLast year COVID-19 kept us apart. This year we are finally together again. \n\nTonight, we meet as Democrats Republicans and Independents. But most importantly as Americans. \n\nWith a duty to one another to the American people to the Constitution. \n\nAnd with an unwavering resolve that freedom will always triumph over tyranny.', metadata={'source': '/content/drive/MyDrive/Dataset/text/state_of_the_union.txt'}),
 Document(page_content='Six days ago, Russia’s Vladimir Putin sought to shake the foundations of the free world thinking he could make it bend to his menacing ways. But he badly miscalculated. \n\nHe thought he could roll into Ukraine and the world would roll over. Instead he met a wall of strength he never imagined. \n\nHe met the Ukrainian people. \n\nFrom President Zelenskyy to every U

In [68]:

from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import FAISS

In [69]:
from google.colab import userdata
OPENAI_API_KEY=userdata.get('OPENAI_API_KEY')

In [70]:

import os
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

In [71]:

embeddings=OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


In [72]:
vectorstore=FAISS.from_documents(text_chunks, embeddings)

In [73]:
retriever=vectorstore.as_retriever()

In [74]:

from langchain.prompts import ChatPromptTemplate

In [75]:
template="""You are an assistant for question-answering tasks.
Use the following pieces of retrieved context to answer the question.
If you don't know the answer, just say that you don't know.
Use ten sentences maximum and keep the answer concise.
Question: {question}
Context: {context}
Answer:
"""

In [76]:
prompt=ChatPromptTemplate.from_template(template)

In [77]:

from langchain.chat_models import ChatOpenAI
from langchain.schema.runnable import RunnablePassthrough
from langchain.schema.output_parser import StrOutputParser

In [78]:
output_parser=StrOutputParser()

In [79]:

llm_model=ChatOpenAI(openai_api_key=OPENAI_API_KEY,model_name="gpt-3.5-turbo")

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


In [80]:

rag_chain = (
    {"context": retriever,  "question": RunnablePassthrough()}
    | prompt
    | llm_model
    | output_parser
)


In [81]:
rag_chain.invoke("How is the United States supporting Ukraine economically and militarily?")

'The United States is supporting Ukraine economically and militarily by providing military assistance, economic assistance, and humanitarian assistance. They have given more than $1 billion in direct assistance to Ukraine. US forces are not engaged and will not engage in conflict with Russian forces in Ukraine. Democracies are rising to the moment in the battle between democracy and autocracy, with the world choosing the side of peace and security. The NATO Alliance was created to secure peace and stability in Europe after World War 2. Ukrainian Americans have a deep bond connecting the two nations, and the US stands with the Ukrainian people. Citizens in Ukraine are bravely defending their homeland, with groups blocking tanks and individuals from all walks of life joining the fight. President Zelenskyy said that "Light will win over darkness" in his speech to the European Parliament. The US is sending an unmistakable signal of support to Ukraine and the world.'